# Cell 1: Markdown
"""
# mRNA Classification Project
**Course:** MSI5001 - Introduction to AI  
**Team Members:** Lisa Mithani, Shawn Lee, Aishwarya Nair, and Kalyani Vijay
**Dataset:** mRNA Classification (Medium difficulty)  
**Objective:** Classify RNA sequences as mRNA vs. other RNA types using machine learning models

---

## Table of Contents
1. Data Loading & Merging
2. Exploratory Data Analysis
3. Feature Extraction
4. Model Training
5. Evaluation & Insights
"""

In [8]:
!pip install biopython
!pip install imbalanced-learn
!pip install torch torchvision torchaudio
!pip install matplotlib
!pip install scikit-learn

In [9]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")


✓ Libraries imported successfully!


---
# 1. Data Loading and Preparation
**Responsible:** Lisa Mithani

## Overview
This section loads the training and testing datasets, merges them, and prepares features and labels for modeling.

## Tasks:
1. Load training.fa (FASTA) and training.csv → merge using inner join
2. Load testing.csv
3. Separate features (X) and labels (y) for train/test sets

---


In [19]:
# ============================================================================
# STEP 1: Load training.fa and training.csv, then merge using inner join
# ============================================================================

# Function to load FASTA file
def load_fasta_to_dataframe(fasta_file):
    """
    Reads a FASTA file and converts it to a pandas DataFrame.
    
    Parameters:
    - fasta_file: path to .fa file
    
    Returns:
    - DataFrame with columns: ['sequence_id', 'sequence']
    """
    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append({
            'sequence_id': record.id,
            'sequence': str(record.seq)
        })
    return pd.DataFrame(sequences)

# Load training.fa (FASTA file)
print("Loading training.fa...")
fasta_df = load_fasta_to_dataframe('dataset/training.fa')
print(f"✓ Loaded {fasta_df.shape[0]} sequences from training.fa")

# Load training.csv
print("\nLoading training.csv...")
csv_df = pd.read_csv('dataset/training_class.csv')
print(f"✓ Loaded training.csv with shape {csv_df.shape}")

# Merge using inner join with DIFFERENT column names
# FASTA has 'sequence_id', CSV has 'name'
print("\nMerging datasets using inner join...")
training_data = pd.merge(
    fasta_df,
    csv_df,
    left_on='sequence_id',   # Column in FASTA
    right_on='name',          # Column in CSV (different name!)
    how='inner'
)

print(f"✓ Merged training data: {training_data.shape}")
print(f"  Columns: {list(training_data.columns)}")
print(f"\n✓ STEP 1 COMPLETE")

# Display first few rows
training_data.head(2)



Loading training.fa...
✓ Loaded 22867 sequences from training.fa

Loading training.csv...
✓ Loaded training.csv with shape (22867, 2)

Merging datasets using inner join...
✓ Merged training data: (14286, 4)
  Columns: ['sequence_id', 'sequence', 'name', 'class']

✓ STEP 1 COMPLETE


,sequence_id,sequence,name,class
0,ENSDART00000138379,TCAAANGGAAAATAATATGTCAGYTGTGATTTTTACTCGANTTAAT...,ENSDART00000138379,1
1,ENSDART00000075994,ATGTCTCTTTTTGAAATAAAAGACCTGNTTNGAGAAGGAAGCTATG...,ENSDART00000075994,1


In [20]:
# ============================================================================
# STEP 2: Load testing.csv
# ============================================================================

print("Loading testing.csv...")
testing_data = pd.read_csv('dataset/test.csv')

print(f"✓ Loaded testing.csv: {testing_data.shape}")
print(f"  Columns: {list(testing_data.columns)}")
print(f"\n✓ STEP 2 COMPLETE")

testing_data.head(2)


Loading testing.csv...
✓ Loaded testing.csv: (4416, 3)
  Columns: ['name', 'sequence', 'class']

✓ STEP 2 COMPLETE


,name,sequence,class
0,TCONS_00059596,CUAAUCCCCCCUCCUCCCGCUCCCGCACCAAAGAGUUGCGCCGCCU...,1
1,TCONS_00059678,CUAUUCGGCGCAGUUGCUAUACGUACCCCAGCCUCGUACACAACGC...,1


In [23]:
# ============================================================================
# STEP 3: Drop class labels and store them in y_train and y_test
# ============================================================================

label_column = 'class'

# --- Training Set ---
print("Separating training data...")
y_train = training_data[label_column].copy()
# Keep 'sequence' column for next teammate to extract features
X_train = training_data.drop(columns=[label_column, 'sequence_id', 'name']).copy()

print(f"✓ X_train shape: {X_train.shape}")
print(f"  Columns: {list(X_train.columns)}")  # Should show ['sequence']
print(f"✓ y_train shape: {y_train.shape}")
print(f"  Class distribution:\n{y_train.value_counts()}")

# --- Testing Set ---
print("\nSeparating testing data...")
y_test = testing_data[label_column].copy()
# Keep 'sequence' column for next teammate
X_test = testing_data.drop(columns=[label_column, 'name']).copy()

print(f"✓ X_test shape: {X_test.shape}")
print(f"  Columns: {list(X_test.columns)}")  # Should show ['sequence']
print(f"✓ y_test shape: {y_test.shape}")
print(f"  Class distribution:\n{y_test.value_counts()}")

print(f"\n✓ STEP 3 COMPLETE")
print(f"\n⚠️ Note: X_train and X_test contain raw sequences.")
print(f"   Next step: Feature extraction (k-mer generation)")



Separating training data...
✓ X_train shape: (14286, 1)
  Columns: ['sequence']
✓ y_train shape: (14286,)
  Class distribution:
class
0    9224
1    5062
Name: count, dtype: int64

Separating testing data...
✓ X_test shape: (4416, 1)
  Columns: ['sequence']
✓ y_test shape: (4416,)
  Class distribution:
class
1    2208
0    2208
Name: count, dtype: int64

✓ STEP 3 COMPLETE

⚠️ Note: X_train and X_test contain raw sequences.
   Next step: Feature extraction (k-mer generation)


---
## Section 1 Complete ✓

**Outputs:**
- `X_train` (14,286 samples × 1 column): Raw RNA sequences
- `y_train` (14,286 labels): Class labels (0 or 1)
- `X_test` (4,416 samples × 1 column): Raw RNA sequences
- `y_test` (4,416 labels): Class labels (0 or 1)

**Note for Next Teammate:**
The `X_train` and `X_test` DataFrames contain raw RNA sequences in the `'sequence'` column. Please convert these to numeric features (e.g., k-mer frequencies) before model training.

**Next Section:** Feature Extraction / Exploratory Data Analysis

---



---
# 2. Feature Extraction
**Responsible:** [Teammate Names TBD]

## Overview
This section converts raw RNA sequences into three types of numeric features for modeling.

We will extract:
1. Character Positional Embeddings
2. Character Tokenization
3. K-mer Frequency Features

Each feature type will be used to train separate models, then compared.

---


### Step 2.1: Character Positional Embedding
Convert RNA sequences to positional embeddings using learned vectors.


In [ ]:
# ============================================================================
# STEP 2.1: Character Positional Embedding - TO BE COMPLETED
# ============================================================================

# TODO: Convert X_train['sequence'] and X_test['sequence'] to positional embeddings
# Output: X_train_embedding, X_test_embedding (numeric DataFrames)

# Example approach:
# - Map each nucleotide (A, U, G, C) to a position index
# - Create embedding vectors for each position
# - Concatenate or average embeddings across sequence length

print("⚠️ Step 2.1 incomplete - waiting for implementation")


---
### Step 2.2: Character Tokenization
Tokenize sequences into character-level representations.


In [ ]:
# ============================================================================
# STEP 2.2: Character Tokenization - TO BE COMPLETED
# ============================================================================

# TODO: Tokenize X_train['sequence'] and X_test['sequence']
# Output: X_train_tokens, X_test_tokens (numeric DataFrames)

# Example approach:
# - Create vocabulary: {'A': 0, 'U': 1, 'G': 2, 'C': 3}
# - Convert each sequence to list of token IDs
# - Pad sequences to same length
# - Convert to numeric array

print("⚠️ Step 2.2 incomplete - waiting for implementation")


---
### Step 2.3: K-mer Feature Extraction
Extract k-mer frequency features (k=3, 4, or 5).


In [ ]:
# ============================================================================
# STEP 2.3: K-mer Feature Extraction - TO BE COMPLETED
# ============================================================================

# TODO: Extract k-mer features from X_train['sequence'] and X_test['sequence']
# Output: X_train_kmers, X_test_kmers (numeric DataFrames)

# OPTION A: Extract k-mers manually
# - Loop through sequences and extract overlapping k-mers
# - Count k-mer frequencies
# - Normalize to create feature vectors

# OPTION B: Load pre-computed features (FASTER)
# X_train_kmers = pd.read_csv('dataset/train_kmer_features_k3.csv')

print("⚠️ Step 2.3 incomplete - waiting for implementation")


---
## Section 2 Summary

**Expected Outputs:**
- `X_train_embedding` (14,286 samples × embedding_dim features)
- `X_test_embedding` (4,416 samples × embedding_dim features)
- `X_train_tokens` (14,286 samples × sequence_length features)
- `X_test_tokens` (4,416 samples × sequence_length features)
- `X_train_kmers` (14,286 samples × n_kmers features)
- `X_test_kmers` (4,416 samples × n_kmers features)

All features are now numeric and ready for class balancing.

---


# 3. Class Balancing
**Responsible:** Shawn Lee

## Overview
Balance class distribution for each feature type separately.

Current training class distribution:
- Class 0: 9,224 samples (64.5%)
- Class 1: 5,062 samples (35.5%)

---


In [ ]:
# Creates a function that will balance the databset passed in
# Input: X_train, y_train
# Output: X_train_balanced, y_train_balanced

# Using SMOTE:
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(random_state=42)
# Usage: X_train_embedding_balanced, y_train_balanced = smote.fit_resample(X_train_embedding, y_train)

def balance_features(X, y):
    smote = SMOTE(random_state=42)
    X_balanced, y_balanced = smote.fit_resample(X, y)
    print(f"✓ Balanced features: {X_balanced.shape}, {y_balanced.shape}")
    return X_balanced, y_balanced

### Step 3.1: Balance Embedding Features COMPLETE ✓
Apply class balancing to positional embedding features.


In [ ]:
# ============================================================================
# STEP 3.1: Balance Embedding Features - COMPLETED
# ============================================================================

# TODO: Balance X_train_embedding with y_train
# New datasets: X_train_embedding_balanced, y_train_balanced

#X_train_embedding_balanced, y_train_balanced = balance_features(X_train_embedding, y_train)
#print("✓ Step 3.1 complete - Embedding features balanced.")

print("⚠️ Step 3.2 incomplete - waiting for dataset to be available.")


---
### Step 3.2: Balance Token Features
Apply class balancing to tokenization features.


In [ ]:
# ============================================================================
# STEP 3.2: Balance Token Features - TO BE COMPLETED
# ============================================================================

# TODO: Balance X_train_tokens with y_train
# New datasets: X_train_tokens_balanced

#X_train_tokens_balanced, y_train_balanced = balance_features(X_train_tokens, y_train)
#print("✓ Step 3.2 complete - Token features balanced.")

print("⚠️ Step 3.2 incomplete - waiting for dataset to be available.")


---
### Step 3.3: Balance K-mer Features
Apply class balancing to k-mer frequency features.


In [ ]:
# ============================================================================
# STEP 3.3: Balance K-mer Features - TO BE COMPLETED
# ============================================================================

# TODO: Balance X_train_kmers with y_train
# New datasets: X_train_kmers_balanced

#X_train_kmers_balanced, y_train_balanced = balance_features(X_train_kmers, y_train)
#print("✓ Step 3.3 complete - K-mer features balanced.")

print("⚠️ Step 3.3 incomplete - waiting for dataset to be available.")


---
## Section 3 Summary

All three feature types are now balanced and ready for modeling.

**Balanced Datasets:**
- Embedding features: X_train_embedding_balanced, y_train_balanced
- Token features: X_train_tokens_balanced, y_train_balanced
- K-mer features: X_train_kmers_balanced, y_train_balanced

---


---
# 4. Model Training
**Responsible:** [Teammate Names TBD]

## Overview
Train three separate logistic regression models using each feature type:
1. Model A: Using Positional Embedding features
2. Model B: Using Tokenization features
3. Model C: Using K-mer features

Each model will be evaluated independently to compare feature effectiveness.

---


### Step 4.1: Train Logistic Regression Model A (Embedding Features)
Train logistic regression on positional embedding features.


In [ ]:
# ============================================================================
# STEP 4.1: Train Model A (Embedding Features) - TO BE COMPLETED
# ============================================================================

# TODO: Train logistic regression on X_train_embedding_balanced
# from sklearn.linear_model import LogisticRegression
# 
# model_A = LogisticRegression(max_iter=1000, random_state=42)
# model_A.fit(X_train_embedding_balanced, y_train_balanced)
# 
# # Predict on test set
# y_pred_A = model_A.predict(X_test_embedding)
#
# Output: model_A, y_pred_A

print("⚠️ Step 4.1 incomplete - Model A training pending")


---
### Step 4.2: Train Logistic Regression Model B (Token Features)
Train logistic regression on character tokenization features.


In [ ]:
# ============================================================================
# STEP 4.2: Train Model B (Token Features) - TO BE COMPLETED
# ============================================================================

# TODO: Train logistic regression on X_train_tokens_balanced
# model_B = LogisticRegression(max_iter=1000, random_state=42)
# model_B.fit(X_train_tokens_balanced, y_train_balanced)
#
# y_pred_B = model_B.predict(X_test_tokens)
#
# Output: model_B, y_pred_B

print("⚠️ Step 4.2 incomplete - Model B training pending")


---
### Step 4.3: Train Logistic Regression Model C (K-mer Features)
Train logistic regression on k-mer frequency features.


In [ ]:
# ============================================================================
# STEP 4.3: Train Model C (K-mer Features) - TO BE COMPLETED
# ============================================================================

# TODO: Train logistic regression on X_train_kmers_balanced
# model_C = LogisticRegression(max_iter=1000, random_state=42)
# model_C.fit(X_train_kmers_balanced, y_train_balanced)
#
# y_pred_C = model_C.predict(X_test_kmers)
#
# Output: model_C, y_pred_C

print("⚠️ Step 4.3 incomplete - Model C training pending")


---
## Section 4 Summary

**Trained Models:**
- Model A: Logistic Regression on Embedding Features
- Model B: Logistic Regression on Token Features
- Model C: Logistic Regression on K-mer Features

All models trained and ready for evaluation.

---


# 5. Classification Performance Evaluation
**Responsible:** Shawn Lee

## Overview
Evaluate and compare the performance of all three logistic regression models.

## Metrics:
- Accuracy
- Precision
- Recall
- F1-Score
- MCC
- Classification Report
- Confusion Matrix
- ROC-AUC

---


In [ ]:
# ============================================================================
# Creation of evaluation functions
# Usage: test_metrics = compute_metrics(y_true, y_pred)
#        print_metrics(test_metrics, model_name, trained_model)
# ============================================================================

def compute_metrics(y_true, y_pred):
    """
    Compute sensitivity, specificity, and MCC for binary classification.
    
    Args:
        y_true: True labels (0 or 1)
        y_pred: Predicted labels (0 or 1)
        
    Returns:
        Dictionary with sensitivity, specificity, mcc, tp, tn, fp, fn, precision, f1-score, recall
    """
    # Compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Sensitivity (Recall, True Positive Rate)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    # Specificity (True Negative Rate)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    
    # Matthews Correlation Coefficient
    mcc = matthews_corrcoef(y_true, y_pred)

    # Calculate Precision and F1-score and Recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0.0
    recall = sensitivity

    return {
        'y_true': y_true,
        'y_pred': y_pred,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'mcc': mcc,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'precision': precision,
        'f1_score': f1,
        'recall': recall
    }

def plot_confusion_matrix(y_true, y_pred):
    """
    Plot confusion matrix using matplotlib.
    
    Args:
        y_true: True labels (0 or 1)
        y_pred: Predicted labels (0 or 1)
    """
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, cmap='Blues')

    # Show all ticks and label them
    ax.set_xticks(np.arange(cm.shape[1]))
    ax.set_yticks(np.arange(cm.shape[0]))
    ax.set_xticklabels(['0', '1'])
    ax.set_yticklabels(['0', '1'])

    # Loop over data dimensions and create text annotations
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            color = "white" if cm[i, j] > cm.max() / 2 else "black"
            ax.text(j, i, cm[i, j], ha="center", va="center", color=color)

    ax.set_xlabel('Predicted label')
    ax.set_ylabel('True label')
    ax.set_title('Confusion Matrix')
    plt.colorbar(im)
    plt.show()

def plot_ROC_Curve(y_true, y_scores):
    """
    Plot an interpretable ROC Curve with AUC and the threshold that maximizes MCC.

    Args:
        y_true: True labels (0 or 1)
        y_scores: Predicted probabilities for the positive class

    Returns:
        (fig, ax, auc, best_t, best_mcc) - useful for further programmatic use
    """
    fpr, tpr, roc_thresholds = roc_curve(y_true, y_scores)
    auc_score = roc_auc_score(y_true, y_scores)

    # Search thresholds (include 0/1 extremes) and pick best by MCC
    search_thresholds = np.linspace(0.0, 1.0, 101)
    best_mcc = -1.0
    best_t = 0.5
    for t in search_thresholds:
        preds_t = (y_scores >= t).astype(int)
        try:
            mcc_t = matthews_corrcoef(y_true, preds_t)
        except Exception:
            mcc_t = -1.0
        if mcc_t > best_mcc:
            best_mcc = mcc_t
            best_t = t

    # Compute FPR/TPR for best threshold (from confusion matrix)
    cm = confusion_matrix(y_true, (y_scores >= best_t).astype(int), labels=[0, 1])
    if cm.size == 4:
        tn, fp, fn, tp = cm.ravel()
        fpr_best = fp / (fp + tn) if (fp + tn) > 0 else 0.0
        tpr_best = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    else:
        # fallback
        fpr_best, tpr_best = 0.0, 0.0

    # Plot
    fig, ax = plt.subplots(figsize=(7, 6))
    ax.plot(fpr, tpr, lw=2, label=f'ROC curve (AUC = {auc_score:.4f})', color='tab:blue')
    ax.fill_between(fpr, tpr, alpha=0.15, color='tab:blue')
    ax.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random (AUC=0.5)')

    # Mark and annotate best threshold point
    ax.scatter([fpr_best], [tpr_best], color='red', zorder=5)
    ax.annotate(f't={best_t:.3f}\nMCC={best_mcc:.3f}\nTPR={tpr_best:.3f}\nFPR={fpr_best:.3f}',
                xy=(fpr_best, tpr_best),
                xytext=(fpr_best + 0.05, tpr_best - 0.12),
                bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.9),
                arrowprops=dict(arrowstyle='->', lw=1),
                fontsize=9)

    # Formatting for interpretability
    ax.set_xlim([-0.01, 1.01])
    ax.set_ylim([-0.01, 1.01])
    ax.set_xlabel('False Positive Rate (1 - Specificity)')
    ax.set_ylabel('True Positive Rate (Sensitivity / Recall)')
    ax.set_title('Receiver Operating Characteristic (ROC) Curve')
    ax.grid(True, linestyle=':', alpha=0.6)
    ax.legend(loc='lower right', framealpha=0.9)
    plt.tight_layout()
    plt.show()

    return fig, ax, auc_score, best_t, best_mcc

def print_metrics(test_metrics, model_name, trained_model=None):
    """
    Print the evaluation metrics in a formatted way.
    
    Args:
        metrics: Dictionary of metrics
        model_name: Name of the model being evaluated (string)
        trained_model: (optional) trained model to get predicted probabilities
    """
    print("=" * 60)
    print(f"{model_name} MODEL EVALUATION")
    print("=" * 60)

    print("\n" + "=" * 60)
    print("MODEL PERFORMANCE")
    print("=" * 60)
    print(f"Accuracy:                  {test_metrics['accuracy']:.4f}")
    print(f"Specificity (Precision/TNR):        {test_metrics['specificity']:.4f}")
    print(f"Sensitivity (Recall/TPR): {test_metrics['sensitivity']:.4f}")
    print(f"F1 Score:                {test_metrics['f1_score']:.4f}")
    print(f"MCC:                      {test_metrics['mcc']:.4f}")
    print("=" * 60)
    print("Classification Report:")
    print(classification_report(test_metrics['y_true'], test_metrics['y_pred']))
    print("=" * 60)
    print(f"\nConfusion Matrix:")
    print(f"  True Positives  (TP): {test_metrics['tp']:5d}")
    print(f"  True Negatives  (TN): {test_metrics['tn']:5d}")
    print(f"  False Positives (FP): {test_metrics['fp']:5d}")
    print(f"  False Negatives (FN): {test_metrics['fn']:5d}")
    print(f"  Precision:               {test_metrics['precision']:.4f}")
    print(f"  F1 Score:                {test_metrics['f1_score']:.4f}")
    print(f"  Recall:                  {test_metrics['recall']:.4f}")
    print(f"  Total Samples:        {test_metrics['tp'] + test_metrics['tn'] + test_metrics['fp'] + test_metrics['fn']:5d}")

    # Plot confusion matrix
    plot_confusion_matrix(test_metrics['y_true'], test_metrics['y_pred'])
    # Plot ROC Curve
    test_probs = trained_model.predict_proba(X_test)[:, 1] if trained_model else test_metrics['y_pred']
    plot_ROC_Curve(test_metrics['y_true'], test_probs)

### Step 5.1: Evaluate All Models
Calculate performance metrics for Models A, B, and C.


In [ ]:
# ============================================================================
# STEP 5.1: Evaluate All Models - TO BE COMPLETED
# ============================================================================

# TODO: Evaluate each model's performance
#
# # Evaluate Model A
# test_metrics_A = compute_metrics(y_test, y_pred_A)
# print_metrics(test_metrics_A, "Model A (Embedding)", trained_model=model_A)
#
# # Repeat for Models B and C
#
# # Create comparison DataFrame
# results = pd.DataFrame({
#     'Model': ['Model A (Embedding)', 'Model B (Tokens)', 'Model C (K-mers)'],
#     'Accuracy': [acc_A, acc_B, acc_C],
#     'Precision': [precision_A, precision_B, precision_C],
#     'Recall': [recall_A, recall_B, recall_C],
#     'F1-Score': [f1_A, f1_B, f1_C]
# })
#
# print(results)

print("⚠️ Step 5.1 incomplete - Model evaluation pending trained models")


---
### Step 5.2: Compare Model Performance
Visualize and compare performance across all models.


In [ ]:
# ============================================================================
# STEP 5.2: Compare Model Performance - TO BE COMPLETED
# ============================================================================

# TODO: Create comparison visualizations
# import matplotlib.pyplot as plt
# import seaborn as sns
#
# # Bar chart comparing metrics
# results.set_index('Model')[['Accuracy', 'Precision', 'Recall', 'F1-Score']].plot(kind='bar', figsize=(10,6))
# plt.title('Model Performance Comparison')
# plt.ylabel('Score')
# plt.legend(loc='lower right')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


print("⚠️ Step 5.2 incomplete - Performance comparison pending trained models")

---
### Step 5.3: Select Best Model
Identify the best-performing model based on evaluation metrics.


In [ ]:
# ============================================================================
# STEP 5.3: Select Best Model - TO BE COMPLETED
# ============================================================================

# TODO: Determine best model
# best_model_idx = results['F1-Score'].idxmax()
# best_model_name = results.loc[best_model_idx, 'Model']
# best_f1 = results.loc[best_model_idx, 'F1-Score']
#
# print(f"\n{'='*60}")
# print(f"BEST MODEL: {best_model_name}")
# print(f"F1-Score: {best_f1:.4f}")
# print(f"{'='*60}")

print("⚠️ Step 5.3 incomplete - Best model selection pending trained models")


---
## Section 5 Summary

**Performance Comparison Complete:**
- All three models evaluated using standard classification metrics
- Best performing model identified
- Results ready for reporting

---


---
# 6. Advanced Model Experiments
**Team Contributions:** Individual advanced model implementations

## Overview
After baseline logistic regression comparison, each team member explores an advanced model with custom preprocessing to maximize performance.

## Team Member Assignments:
- **Lisa:** Random Forest Classifier
- **Shawn:** Recurrent Neural Network (RNN)
- **Aishwarya:** Long Short-Term Memory (LSTM)
- **Kalyani:** Transformer Model

Each subsection contains: Preprocessing → Model Training → Performance Evaluation (all in one cell).

---


## 6.1 Random Forest Classifier
**Responsible:** Lisa

### Approach
Use Random Forest with k-mer features and custom preprocessing.

This cell contains: Additional preprocessing → Model training → Performance evaluation.


---
## 6.2 Recurrent Neural Network (RNN)
**Responsible:** Shawn

### Approach
Use RNN with sequential token features to capture sequence patterns.

This cell contains: Additional preprocessing → Model training → Performance evaluation.

1st version of codes in this cell was generated usig ChatGPT with prompt "provide codes to train a RNN model to classify RNA dataset in python assuming that the dataset that requires tokenization and class balancing". Subsequently, changes were done to ensure correctness and alignment to codes in other cells in this project.


In [ ]:
# ============================================================================
# SECTION 6.2: Recurrent Neural Network (RNN) MODEL TRAINING
# ============================================================================

print("="*60)
print("SECTION 6.2: RNN MODEL TRAINING")
print("="*60)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✓ Using device: {device}")

#### STARTING FROM HERE, CODES TO BE REMOVED ONCE TOKENIZATION IS DONE ABOVE. TAKE NOTE TO USE BALANCED DATASET ####

print("\n[Step 1/4] Tokenizing sequences...")

# Define vocabulary for RNA nucleotides
PAD_IDX = 0
UNK_IDX = 1
nucleotides = sorted(list(set('ACGTURYSWKMBDHVN')))  # IUPAC alphabet
char2idx = {ch: i + 2 for i, ch in enumerate(nucleotides)}
char2idx['<PAD>'] = PAD_IDX
char2idx['<UNK>'] = UNK_IDX
vocab_size = len(char2idx)

print(f"✓ Vocabulary size: {vocab_size}")

def tokenize_sequence(seq, char2idx, max_len=512):
    """Convert sequence string to list of token indices."""
    tokens = [char2idx.get(char, UNK_IDX) for char in seq[:max_len]]
    # Pad to max_len
    if len(tokens) < max_len:
        tokens = tokens + [PAD_IDX] * (max_len - len(tokens))
    return tokens

# Determine max sequence length (cap at 512 for memory)
max_seq_len = min(512, max(X_train['sequence'].str.len().max(), 
                             X_test['sequence'].str.len().max()))
print(f"✓ Max sequence length: {max_seq_len}")

# Tokenize all sequences (assigning to X_train_tokens & X_test_tokens to align with unfinished codes above. To update again if variable name changes.)
X_train_tokens_balanced = X_train['sequence'].apply(
    lambda s: tokenize_sequence(s, char2idx, max_seq_len)
).tolist()
X_test_tokens = X_test['sequence'].apply(
    lambda s: tokenize_sequence(s, char2idx, max_seq_len)
).tolist()

y_train_balanced = y_train  # Placeholder until balancing is done above
#### END OF REMOVAL ####

# Convert to tensors
X_train_tensor = torch.tensor(X_train_tokens_balanced, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_tokens, dtype=torch.long)
y_train_tensor = torch.tensor(y_train_balanced.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

print(f"✓ Training tensor shape: {X_train_tensor.shape}")
print(f"✓ Test tensor shape: {X_test_tensor.shape}")


# ----------------------------------------------------------------------------
# Create DataLoaders
# ----------------------------------------------------------------------------
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"✓ Train batches: {len(train_loader)}, Test batches: {len(test_loader)}")

# ----------------------------------------------------------------------------
# Define RNN Model
# ----------------------------------------------------------------------------

class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_size=128, 
                 num_layers=2, dropout=0.3, pad_idx=PAD_IDX):
        super(RNNClassifier, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        # RNN layer (GRU for better gradient flow)
        self.rnn = nn.GRU(
            embedding_dim, 
            hidden_size, 
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size * 2, 64)  # *2 for bidirectional
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, 2)  # Binary classification
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # x: (batch, seq_len)
        embedded = self.embedding(x)  # (batch, seq_len, embed_dim)
        
        # RNN forward pass
        rnn_out, hidden = self.rnn(embedded)  # rnn_out: (batch, seq_len, hidden*2)
        
        # Use last hidden state (concatenate forward and backward)
        if self.rnn.bidirectional:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)  # (batch, hidden*2)
        else:
            hidden = hidden[-1]  # (batch, hidden)
        
        # Classification head
        out = self.relu(self.fc1(hidden))
        out = self.dropout(out)
        out = self.fc2(out)
        
        return out

# Instantiate model
model = RNNClassifier(
    vocab_size=vocab_size,
    embedding_dim=64,
    hidden_size=128,
    num_layers=2,
    dropout=0.3,
    pad_idx=PAD_IDX
).to(device)

print(f"✓ Model created with {sum(p.numel() for p in model.parameters()):,} parameters")

# ----------------------------------------------------------------------------
# Training Setup
# ----------------------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

# ----------------------------------------------------------------------------
# Training Loop
# ----------------------------------------------------------------------------
print("\nTraining RNN model...")

num_epochs = 10
best_val_acc = 0.0
train_losses, val_accs = [], []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    epoch_loss = 0.0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    # Validation phase
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
    
    val_acc = correct / total
    val_accs.append(val_acc)
    
    # Learning rate scheduling
    scheduler.step(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_rnn_model.pth')
    
    print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f}")

print(f"\n✓ Training complete! Best validation accuracy: {best_val_acc:.4f}")

# Load best model
model.load_state_dict(torch.load('best_rnn_model.pth'))

# ----------------------------------------------------------------------------
# Evaluation on Test Set
# ----------------------------------------------------------------------------
print("\n" + "="*60)
print("RNN MODEL EVALUATION")
print("="*60)

model.eval()
all_preds = []
all_probs = []

with torch.no_grad():
    for batch_X, _ in test_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())  # Probability of class 1

y_pred_rnn = np.array(all_preds)
y_probs_rnn = np.array(all_probs)

# Compute metrics using existing function
test_metrics_rnn = compute_metrics(y_test.values, y_pred_rnn)
test_metrics_rnn['accuracy'] = (test_metrics_rnn['tp'] + test_metrics_rnn['tn']) / len(y_test)

# Print detailed metrics
print_metrics(test_metrics_rnn, "RNN", trained_model=None)

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(train_losses, label='Training Loss', color='tab:blue')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss Over Time')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(val_accs, label='Validation Accuracy', color='tab:orange')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Validation Accuracy Over Time')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("✓ RNN MODEL SECTION COMPLETE")
print("="*60)

---
## 6.3 Long Short-Term Memory (LSTM)
**Responsible:** Aishwarya

### Approach
Use LSTM to capture long-range dependencies in RNA sequences.

This cell contains: Additional preprocessing → Model training → Performance evaluation.


---
## 6.4 Transformer Model
**Responsible:** Kalyani

### Approach
Use Transformer architecture with attention mechanism for sequence classification.

This cell contains: Additional preprocessing → Model training → Performance evaluation.


---
## 6.5 Comprehensive Model Comparison
Compare all models: Baseline Logistic Regression + Advanced Models


---
## Section 6 Complete ✓

**Advanced Models Implemented:**
- ✅ Random Forest (Lisa)
- ✅ RNN (Shawn)
- ✅ LSTM (Aishwarya)
- ✅ Transformer (Kalyani)
- ✅ Comprehensive comparison of all 7 models

**Key Findings:** [To be filled after implementation]

---
